<a href="https://colab.research.google.com/github/ashishsalunkhe/Closed-Domain-Question-Answering/blob/master/cdqa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install cdqa

     |████████████████████████████████| 61kB 2.0MB/s 
     |████████████████████████████████| 133kB 8.3MB/s 
     |████████████████████████████████| 645kB 50.2MB/s 
  Created wheel for cdqa: filename=cdqa-1.3.5-cp36-none-any.whl size=59618 sha256=e30242e8e3229dc67dce59482b59e295634c2d347357e7a1629b6759b4225c1e
  Stored in directory: /root/.cache/pip/wheels/96/7e/86/505f87a5edf9f69a11e296b877bfb985904a1ed6c85e0892d0
  Created wheel for tika: filename=tika-1.19-cp36-none-any.whl size=29224 sha256=b0ae47623747708de0e70e1910e11ad122a10030b6dbe22d1b8f089c9f18fe7c
  Stored in directory: /root/.cache/pip/wheels/b4/db/8a/3a3f0c0725448eaa92703e3dda71e29dc13a119ff6c1036848
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9681 sha256=11d055dfd3938d710bbf3563a0aa8820c87fcfbf782df80f5f8084c4bef2e3a1
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built cdqa tika wget


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import os
import pandas as pd
from ast import literal_eval

from cdqa.utils.filters import filter_paragraphs
from cdqa.pipeline import QAPipeline

/usr/local/lib/python3.6/dist-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [4]:
from cdqa.utils.download import download_model, download_bnpp_data

download_bnpp_data(dir='/content/drive/My Drive/CDQA/bnpp_newsroom_v1.1/')
download_model(model='bert-squad_1.1', dir='/content/drive/My Drive/CDQA/models')


bnpp_newsroom-v1.1.csv already downloaded

bert_qa_vCPU-sklearn.joblib already downloaded


In [0]:
df_qa = pd.read_csv('/content/drive/My Drive/CDQA/politics18.csv')

In [15]:
df_qa.head()

,author,content,date,tag,title,url,website
0,The Quint,"Tulsi Gabbard, the first Hindu lawmaker from H...",2018-11-13,Tulsi Gabbard,Tulsi Gabbard: Hindu War Veteran US Lawmaker M...,https://www.thequint.com/news/world/tulsi-gabb...,quint
1,Pushpesh Pant,Swami Vivekananda ranks among the highest in t...,2018-07-03,"Swami Vivekananda,Hinduism,Tolerance,Politics ...",Swami Vivekananda’s 156th Birth Anniversary 20...,https://www.thequint.com/voices/opinion/swami-...,quint
2,Meeran Chadha Borwankar,"In the news for the wrong reasons, the Central...",2018-07-18,"Modi Government,Central Bureau of Investigatio...",Rift Between CBI Chief & Special Director: Thi...,https://www.thequint.com/voices/opinion/cbi-co...,quint
3,Mayukh Ghosh,Reminiscing about the past is a dreadful tool ...,2018-05-14,"Satyajit Ray,Shyam Benegal,Smita Patil,Politic...",The Late Filmmaker Mrinal Sen’s ‘Cinema of Res...,https://www.thequint.com/entertainment/indian-...,quint
4,Akanksha Kumar,: Akanksha Kumar : Sandeep Suman Twenty-three ...,2018-04-30,"Tihar Jail,Supreme Court of India,Life Impriso...",Tandoor Murder: Former Delhi Police Joint Comm...,https://www.thequint.com/videos/news-videos/ho...,quint


In [12]:
df = pd.read_csv('/content/drive/My Drive/CDQA/bnpp_newsroom_v1.1/bnpp_newsroom-v1.1.csv', converters={'paragraphs': literal_eval})
df = filter_paragraphs(df)
df.head()

,date,title,category,link,abstract,paragraphs
0,13.05.2019,The banking jobs : Assistant Vice President – ...,Careers,https://group.bnpparibas/en/news/banking-jobs-...,Within the Group’s Corporate and Institutional...,[I manage a team in charge of designing and im...
1,13.05.2019,BNP Paribas at #VivaTech : discover the progra...,Innovation,https://group.bnpparibas/en/news/bnp-paribas-v...,"From Thursday 16 to Saturday 18 May 2019, join...","[With François Hollande, Chairman of French fo..."
2,13.05.2019,"""The bank with an IT budget of more than EUR6 ...",Group,https://group.bnpparibas/en/news/the-bank-budg...,"Interview with Jean-Laurent Bonnafé, Director ...","[We did the groundwork between 2012 and 2016, ..."
3,10.05.2019,BNP Paribas at #VivaTech : discover the progra...,Innovation,https://group.bnpparibas/en/news/bnp-paribas-v...,"From Thursday 16 to Saturday 18 May 2019, join...","[As part of the ‘United Tech of Europe’ theme,..."
4,10.05.2019,When Artificial Intelligence participates in r...,Careers,https://group.bnpparibas/en/news/artificial-in...,As the competition to attract talent intensifi...,[Online recruitment is already the norm. Accor...


In [6]:
cdqa_pipeline = QAPipeline(reader='/content/drive/My Drive/CDQA//models/bert_qa_vCPU-sklearn.joblib')
cdqa_pipeline.fit_retriever(df=df)

100%|██████████| 231508/231508 [00:00<00:00, 5788361.05B/s]


QAPipeline(reader=BertQA(bert_model='bert-base-uncased', do_lower_case=True,
                         fp16=False, gradient_accumulation_steps=1,
                         learning_rate=3e-05, local_rank=-1, loss_scale=0,
                         max_answer_length=30, n_best_size=20, no_cuda=False,
                         null_score_diff_threshold=0.0, num_train_epochs=2,
                         output_dir=None, predict_batch_size=8, seed=42,
                         server_ip='', server_port='', train_batch_size=8,
                         verbose_logging=False, version_2_with_negative=False,
                         warmup_proportion=0.1),
           retrieve_by_doc=False,
           retriever=BM25Retriever(b=0.75, floor=None, k1=2.0, lowercase=True,
                                   max_df=0.85, min_df=2, ngram_range=(1, 2),
                                   preprocessor=None, stop_words='english',
                                   token_pattern='(?u)\\b\\w\\w+\\b',
             

In [0]:
query = 'Since when does the Excellence Program of BNP Paribas exist?'
prediction = cdqa_pipeline.predict(query)

In [8]:
print('query: {}'.format(query))
print('answer: {}'.format(prediction[0]))
print('title: {}'.format(prediction[1]))
print('paragraph: {}'.format(prediction[2]))

query: Since when does the Excellence Program of BNP Paribas exist?
answer: January 2016
title: BNP Paribas’ commitment to universities and schools
paragraph: Since January 2016, BNP Paribas has offered an Excellence Program targeting new Master’s level graduates (BAC+5) who show high potential. The aid program lasts 18 months and comprises three assignments of six months each. It serves as a strong career accelerator that enables participants to access high-level management positions at a faster rate. The program allows participants to discover the BNP Paribas Group and its various entities in France and abroad, build an internal and external network by working on different assignments and receive personalized assistance from a mentor and coaching firm at every step along the way.
